In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("../../")
sys.path.append("../")

In [ ]:
from os.path import join, basename
from glob import glob
import numpy as np
import cv2
import matplotlib.pyplot as plt
from typing import List, Any
from matplotlib import cm

from utils import load_syn_images, show_single_image, show_multiple_images
from photometric.estimate_alb_nrm import estimate_alb_nrm
from photometric.check_integrability import check_integrability

In [ ]:
def get_integrability_outputs(image_stack, scriptV):
    [albedo, normals] = estimate_alb_nrm(image_stack, scriptV)
    [p, q, SE] = check_integrability(normals)
    return p, q, SE

With $n =5$

In [ ]:
image_dir = "../SphereGray5/"

In [ ]:
[image_stack, scriptV] = load_syn_images(image_dir)
[h, w, n] = image_stack.shape

In [ ]:
p, q, SE = get_integrability_outputs(image_stack, scriptV)

In [ ]:
threshold = 0.005

In [ ]:
np.sum(SE > threshold)

In [ ]:
SE[SE <= threshold] = float('nan')

In [ ]:
p.shape

In [ ]:
stride = 1

# meshgrid
X, Y, _ = np.meshgrid(np.arange(0,np.shape(p)[0], stride),
np.arange(0,np.shape(p)[1], stride),
np.arange(1))
X = X[..., 0]
Y = Y[..., 0]

# plotting the SE
H = SE[::stride,::stride]
fig = plt.figure(figsize=(10, 10))
ax = fig.gca(projection='3d')
ax.plot_surface(X,Y, H.T, linewidths=0.2)
ax.set_title("Pixels with SE $>$ threshold = 0.005 for $n = 5$", fontsize=25)

plt.savefig("../results/integrability_test_5.png", bbox_inches="tight")

plt.show()

Varying $n$

In [ ]:
image_dir = "../SphereGray25/"
# obtain many images in a fixed view under different illumination
[image_stack, scriptV] = load_syn_images(image_dir)
[h, w, n] = image_stack.shape

In [ ]:
num_images = [1, 2, 3, 6, 9, 12, 15, 18, 21, 25]
all_SE = []
threshold = 0.005

In [ ]:
for n in num_images:
    # get the images
    image_substack = image_stack[:, :, :n].copy()
    scriptV_sub = scriptV[:n, :].copy()
    p, q, SE = get_integrability_outputs(image_substack, scriptV_sub)
    
    all_SE.append(np.sum(SE > threshold))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

ax.set_title("Integrability test with varying $n$")
ax.set_xlabel("Number of images $n$")
ax.set_ylabel("Number of outliers (SE$(x, y)$ > threshold)")

ax.plot(num_images, all_SE, 'o--')
ax.set_xticks(num_images)
ax.grid()

plt.savefig("../results/integrability_test_n.png", bbox_inches="tight")
plt.show()